# Data Wrangling Project: OpenStreetMap
## Discovering Versailles (France) and its surroundings
![chateau](img/799px-Versailles-FacadeJardin.jpg)
---
## Introduction and Map Area

![map](img/map.png)

The map used in this project is from OpenStreetMap, obtained as a custom extract from mapzen.com ([link to dataset](https://s3.amazonaws.com/mapzen.odes/ex_DRcFT2edje1bEvDwVEKv4jZvAAvP8.osm.bz2)).

I lived in this area (in Saint-Germain-En-Laye, precisely) for 7 years before moving to Romania, so I was interested in analysing and possibly improving the data.  
Moreover, Versailles is famous around the world for its stunning and gigantic palace, built in its current configuration by Louis XIV from 1662. I thought it would be an area that the reviewers could relate to and might be interested in discovering in more detail.

**Note**: By default, any code is hidden in the HTML version of this document. However there is a "Show Code" button at the top which allows the reader to see the code that was used to generate any result shown.

## A. Data Cleansing

In this first part of the project, we want to assess and improve the quality of the data.
The full data is 225MB uncompressed. To speed up development time, the code was first tested using a small sample of the data (every 25th top level element), which represents about 9MB.

**Note on efficiency**: The code snippets below have been written to be largely independent from one another and they execute distinct tasks in line with the document structure. It would have been far more efficient from a computational point of view to execute all checks within one iterative pass of the full OSM XML document, rather than separate tasks and execute each in its own pass. However in the context of this project, we felt it was more instructional to have this structured approach because the focus of the project is more on the data wrangling methodology than on the subsequent use of the data.

In [3]:
import sys
import xml.etree.cElementTree as ET
import pprint

### A.1. Validity / Uniformity

Here we want to ensure that the data provided conforms to certain rules and schemas and that values are consistent. More specifically, we will assess the following points:

* Ensure that all elements in the XML file have a limited and consistent set of tags
* Check that latitude and longitude are expressed in a consistent and usable format
* Make sure postcodes respect the French post format
* Ensure that street types are logical and consistently spelled throughout the document.

#### A.1.a. Data tags

We collect and count the unique element tags in the dataset:

In [4]:
filename = './Versailles.osm/Versailles.osm'


def count_tags(filename):
    ''' Identifies unique tag types and counts occurences for each
    In:
        filename (string): Path to OSM XML file to assess
    Out:
        dict: Unique tag types as keys, counts as values   
    '''   
    tags = {}
    context = ET.iterparse(filename, events = ('start',))  # Detect starts only
    for event, elem in context:     # Go through each element in the OSM XLM file
        if not(elem.tag in tags):   # Create new dict key if not already there   
            tags[elem.tag] = 0
        tags[elem.tag] += 1
    return tags

pprint.pprint(count_tags(filename))

{'bounds': 1,
 'member': 24412,
 'nd': 1332859,
 'node': 1007548,
 'osm': 1,
 'relation': 1942,
 'tag': 483337,
 'way': 159884}


Based on the documentation for OSM XML files, these tags appear correct. We also note that the data contains over one million nodes, around 160,000 ways and 480,000 tags, giving us a nice dataset to work on.

#### A.1.b. Latitude / longitude

To make the data consistent and usable, it is desirable to have latitde and longitude expressed as floating point numbers (as opposed to strings such as "48.92N" / "2.13E"). At the same time, we will want to assess that all the element positions are within the bounds of the map extract that we selected (ie. [48.758 - 48.919] / [2.019 - 2.178]).

In [5]:
lat_bounds = (48.7582257, 48.9188896)
lon_bounds = (2.0190811, 2.1783828)

def is_number(s):
    ''' Returns True if string or float s is or can be coerced into a float, 
    False otherwise'''
    try:
        float(s)
        return True
    except ValueError:
        return False
        
def is_within_bounds(f, bounds):
    ''' Checks whether f is within the interval defined by bounds
    In:
        f (float): Value to test
        bounds (tuple): lower, upper bounds of the interval where f might be 
                        included.
    Out:
        bool: True if f is contained in the interval (equality allowed),
              False if f is strictly lower than lower bound or strictly greater
              than upper bound.
    '''
    if f >= bounds[0] and f <= bounds[1]:
        return True
    return False
        
def check_positions(filename):  
    ''' Checks latitude and longitude for validity and accuracy
    In:
        filename (string): Path to the file to check
    Out:
        dict: Values = Counts of the following occurences (used as keys):
            Null -- at least one of latitude and longitude is None
            Empty -- at least one of latitude and longitude is ""
            Non_number -- at least one of latitude and longitude is not a 
                          float or cannot be coerced into a float
            Out_of_bounds -- at least one of latitude and longitude is outside
                             the bounds of the selected map area
            Correct -- none of the above occurs
    '''
    counts = {'Null': 0, 'Empty': 0, 'Non_number': 0, 'Out_of_bounds': 0, 
              'Correct': 0}
    context = ET.iterparse(filename, events = ('start', )) # Detect starts only
    for event, elem in context:
        if elem.tag == 'node':  # Check for each node element
            lat = elem.get('lat').strip()  # Get rid of unwanted spaces
            lon = elem.get('lon').strip()
            if (lat is None or lon is None):
                counts['Null'] += 1
            elif (lat == "" or lon == ""):
                counts['Empty'] += 1
            elif not (is_number(lat) and is_number(lon)):
                counts['Non_number'] += 1
            elif not (is_within_bounds(float(lat), lat_bounds) and 
            is_within_bounds(float(lon), lon_bounds)):  
            # If lat or lon is out of bounds:
                counts['Out_of_bounds'] += 1
            else:
                counts['Correct'] += 1
    return counts
                
pprint.pprint(check_positions(filename))

{'Correct': 1007548,
 'Empty': 0,
 'Non_number': 0,
 'Null': 0,
 'Out_of_bounds': 0}


It turns out that all the positional data appears to be correct. There is probably some screening done at the user input level when updating maps which stops errors from creeping in.

#### A.1.c. Postcode format

At this stage we are not checking if the postcodes make sense, merely if they have the correct format. French postcodes have 5 digits, the first two indicating the department (eg. 78 is for the Yvelines department, where most of our data comes from).

In [6]:
import re

postcode_re = re.compile(r'^\d{5}$')  # Regex match to an isolated string of 
                                      # 5 consecutive digits

def audit_postcodes(filename):
    ''' Checks postocde to ensure every tag with an addr:postcode key has a postcode 
    value and that they follow the standard French postcode format (5 
    consecutive digits)
    In:
        filename (string): Path to the file to check
    Out:
        dict: Values = Counts of the following occurences (used as keys):
            Null -- there is a postcode key but no value (None)
            Empty -- there is a postcode key but its value is ""
            Incorrect -- there is a postcode key but its value does not follow 
                         the convention
            Correct -- none of the above occurs
    '''
    counts = {'Null': 0, 'Empty': 0, 'Incorrect': 0, 'Correct': 0}
    context = ET.iterparse(filename, events = ('start', )) # Detect starts only
    for event, elem in context:
        if elem.tag == 'node':
            for elt in elem.findall('tag'):
                if elt.get('k') == 'addr:postcode':  
                # If the tag contains a postcode field
                    pc = elt.get('v').strip()  # Then get the postcode value
                    if pc == None:
                        counts['Null'] += 1
                    elif pc == "":
                        counts['Empty'] += 1
                    elif not re.match(postcode_re, pc): 
                    # If not in line with the 5-digit convention:
                        counts['Incorrect'] += 1
                    else:
                        counts['Correct'] += 1
    return counts

pprint.pprint(audit_postcodes(filename))

{'Correct': 2365, 'Empty': 0, 'Incorrect': 0, 'Null': 0}


Once again, we seem to be in luck in that all the postcodes are correctly formatted.

#### A.1.d. Street / way types

We will now focus on the different types of ways that exist in the document. Street names are human-edited and therefore prone to inconsistency and errors. The first step is therefore to list all the unique labels for streets and ways. To do this we adapt some code from the Intro to Data Wrangling course on Udacity.com.

One of the adaptations is in the regular expression that is used to look for the street labels. In English, these would normally be positioned after the street name ("Lexington Avenue"), whereas in France, they are before ("Avenue du Général Leclerc").  

Another point to keep in mind is that we need to convert strings to unicode because of special characters used in French but not in English, such as é, è, à, ô, ç etc.

In [7]:
from collections import defaultdict

street_type_re = re.compile(r'^\S+\.?', re.IGNORECASE)  # Regex to find 
# continuous groups of non-whitespace characters, each possibly ending with a .

# List of expected types of streets and ways:
expected = [u"rue", u"avenue", u"boulevard", 
            u"route", u"place", u"villa", 
            u"impasse", u"passage", u"voie", 
            u"square", u"sentier", u"ruelle",
            u"allée", u"chemin", 
            u"rond-point", u"cours",
            u"parc", u"promenade", u"résidence", 
            u"domaine", u"quai", u"cour", u"clos",
            u"autoroute", u"route nationale", 
            u"route départementale", 
            u"route communale", u"hameau"]

def audit_street_type(street_types, street_name):
    ''' For each street name, selects the first word (assumed to be street type) and
    compares with the list of expected street types. 
    In case it is not there:
    The street_types dict (passed by the upper environment) uses the unexpected 
    street type as a key and the full street name is added to the value (which
    is a set).
    
    In: 
        street_types (dict of sets): Keys are unique unexpected street types, 
            values are sets of full street names with these street types. These 
            sets get updated in place by the function.
        street_name (string): The street name to parse and compare with the list
            of expected street types.
    Out:
        Nothing
    '''
    m = street_type_re.search(street_name)
    if m:
        street_type = unicode(m.group())  # Convert to unicode because of French-specific characters
        if street_type.lower() not in expected:
            street_types[street_type].add(street_name)  # Record all streets 
            # with unexpected street types


def is_street_name(elem):
    ''' Checks whether a 'tag' element of the OSM XML file refers to a street name.
    Returns True if this is the case, False if not.
    '''
    return (elem.attrib['k'] == "addr:street")
    

def audit_all_streets(filename):
    ''' Applies audit_street_type to all elements of the OSM XML file containing a
    street name. Returns the collection of unexpected street types in the form 
    of a dictionary where key = unexpected street type, value = set of street
    names with this street type.
    
    In:
        filename (string): Path to the file to check
    Out:
        dict: Keys are unique street types not found in the reference list 
        ("expected"), values are sets containing all unique street names with
        these street types.
    '''
    osm_file = open(filename, "r")
    street_types = defaultdict(set)
    
    for event, elem in ET.iterparse(osm_file, events = ("start",)): 
                                                            # Detect starts only
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):  # Iterate through all "tag" elements
                if is_street_name(tag):   # Audit street type when the element 
                                          # contains a street name
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

street_types = audit_all_streets(filename)
pprint.pprint(street_types.keys())  # Print unexpected street types (without the related street names)

[u'Les',
 u'Ile',
 u'Centre',
 u'C.C.',
 u'Residence',
 u'Le',
 u"Grand'Rue",
 u'CCR',
 u'Otis',
 u'Allee',
 u'A\xe9rodrome',
 u'Grande',
 u'\xc9lys\xe9e',
 u'Mail',
 u'Jean',
 u'Guyancourt']


It turns out that aside from font case issues, the dataset is fairly clean. There are a few "street" labels that actually refer to shopping malls (or similar), an island (on the Seine river) and even to an airfield. The other issues are: missing accents on letters (Allee ilo. Allée), missing steet type ("Jean Macé" ilo. "Rue Jean Macé", "Otis Mygatt" ilo. "Avenue Otis Mygatt") or unusual street labels ("Grand'Rue" and "Grande Rue", where usually "Rue" would come first -- this is unusual but correct).  

Some of these are acceptable but there are few genuine issues to fix:

- 'Centre' (actually 'Centre Commercial Régional', but only the first word was picked up by the regular expression), C.C. and CCR all refer to the same shopping mall (the largest in the area) and should be updated to the full expression 'Centre Commercial Régional'.
- 'Allee', 'Residence': Replace with 'Allée' and 'Résidence' respectively
- 'Aérodrome' (airfield) is not a valid street type and should be stripped (investigating the document, we see that the node is already named 'Aeroclub' so the notion that it relates to airplanes is already present).
- 'Elysée' is incomplete and refers to a "Résidence" (a housing program) named Elysée 2. We will therefore update it to "Résidence Elysée 2".
- 'Otis' refers to 'Avenue Otis Mygatt'
- 'Jean Macé' refers to 'Rue Jean Macé'
- 'Guyancourt' is a town name. From the node IDs and their position, I was able infer that the tag refered to 'Rue Louis Breguet' in Guyancourt.


In [8]:
'''Values to replace and to be repaced with:'''
street_mapping = { u"allee": u"Allée",
            u"Allee": u"Allée",
            u"hameau": u"Hameau",
            u"Residence": u"Résidence",
            u"résidence": u"Résidence",
            u"Centre Commercial Régional": u"Centre Commercial",
            u"Centre commercial": u"Centre Commercial",
            u"C.C.": u"Centre Commercial",
            u"CCR": u"Centre Commercial",
            u"\xc9lys\xe9e 2": u"Résidence \xc9lysée 2",
            u"Aérodrome - ": u"",
            u"Otis": u"Avenue Otis",
            u"Jean Macé": u"Rue Jean Macé",
            u"Guyancourt": u"Rue Louis Breguet"
            }

def update_street_name(street_name, mapping):
    ''' Uses the mapping dictionary to correct street types: For each key in 
    mapping, looks for it in street_name and replaces this string with the 
    associated value (which is the correctly spelled street type).
    
    In: 
        street_name (string): Full street name possibly requiring a correction
        mapping (dictionary): Keys are wrongly spelled street types, values are
            strings to use as replacements within street_name.
    Out:
        string: Correctly spelled street name.
    '''
    for bad_name in mapping:
        match = re.compile('^' + bad_name)
        try:  # Substitute the string bad_name with the associated value in 
              # mapping. The rest of the street_name string is untouched.
            street_name = re.sub(match, mapping[bad_name], street_name)
        except LookupError:  # If this happens, we need to manually amend the 
                             # mapping.
            print "Missing entry in mapping"
            continue
    return street_name

for street_type, ways in street_types.iteritems():  # Do this for each unexpected street type
    for street_name in ways:
        better_name = update_street_name(street_name, street_mapping)
        if street_name != better_name:  # Display replacements:
            print street_name, "=>", better_name

Centre commercial SQY Ouest => Centre Commercial SQY Ouest
Centre commercial parly 2 => Centre Commercial parly 2
C.C. Parly 2 => Centre Commercial Parly 2
Residence de la Roseraie => Résidence de la Roseraie
CCR Parly II => Centre Commercial Parly II
Otis Mygatt => Avenue Otis Mygatt
Allee Pierre de Coubertin => Allée Pierre de Coubertin
Aérodrome - Rue du Docteur Vaillant => Rue du Docteur Vaillant
Élysée 2 => Résidence Élysée 2
Jean Macé => Rue Jean Macé
Guyancourt => Rue Louis Breguet


This confirms that our cleaning function works correctly. We are going to use it on the fly when converting the data into a JSON file later.

### A.2. Accuracy and consistency
In this section, we will assess whether the city and postcode data are complete and correct when compared to a data source that we know to be 100% accurate. The reference file we used is made available by La Poste (main French postal service) and can be downloaded [here](http://datanova.legroupe.laposte.fr/explore/dataset/laposte_hexasmal/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true). The file is in CSV format with semi-column as separators as is the norm with French documents -- commas are used as decimal separators. It lists the 39,000 towns in France.

We first have a look at all the (postcode, city) pairs that exist in the document:

In [9]:
pc_cities = set()
    
with open(filename, 'r') as osm_file:
    for event, elem in ET.iterparse(osm_file, events = ("start",)):
        if elem.tag == "node" or elem.tag == 'way':
            pc_tag = elem.find("./tag[@k='addr:postcode']")  # Use XPath command
            # to locate postcode in node or way
            try:
                elem_pc = pc_tag.attrib['v']
            except AttributeError:  # Case no tag attribute 'v'
                elem_pc = None
            
            city_tag = elem.find("./tag[@k='addr:city']")  # Use XPath command 
            # to locate city in node or way
            try:
                elem_city = city_tag.attrib['v']
            except AttributeError:  # Case no tag attribute 'v'
                elem_city = None
            if not (elem_pc is None and elem_city is None): # As long as at 
            # least one of postcode or city name exists, add to set:
                pc_cities.add((elem_pc, elem_city))

pprint.pprint(pc_cities)
        

set([(None, '78170'),
     (None, 'Bougival'),
     (None, 'Buc'),
     (None, 'Croissy-sur-Seine'),
     (None, 'Guyancourt'),
     (None, 'La Celle-Saint-Cloud'),
     (None, 'Le Chesnay'),
     (None, u'Le V\xe9sinet'),
     (None, 'Marly-le-Roi'),
     (None, 'Montesson'),
     (None, 'Montigny-le-Bretonneux'),
     (None, 'Noisy-le-Roi'),
     (None, 'Roquencourt'),
     (None, u"Saint-Cyr-l'\xc9cole"),
     (None, 'Saint-Germain-en-Laye'),
     (None, 'Versailles'),
     (None, 'Viroflay'),
     (None, 'le Chesnay'),
     ('78000', None),
     ('78000', 'Versailles'),
     ('78100', None),
     ('78100', 'Saint-Germain-en-Laye'),
     ('78101', None),
     ('78103', None),
     ('78110', None),
     ('78110', 'Le Pecq'),
     ('78110', u'Le V\xe9sinet'),
     ('78110', 'Saint-Germain-en-Laye'),
     ('78112', None),
     ('78112', 'Fourqueux'),
     ('78120', 'Viroflay'),
     ('78140', None),
     ('78140', u'V\xe9lizy Villacoublay'),
     ('78140', u'V\xe9lizy-Villacoublay'),
 

We can see that there are many consistency issues around spelling. Some cities have several different postcodes but this is expected (different post offices serve different areas of the same city). In many instances, we have a postcode but no city name. And in some cases, we have a city name but no postcode. We also have a case where a postcode was incorrectly entered as a city name ('78170').  
To fix the majority of these issues, one idea would be to use the La Poste file to look postcodes up and update the city names with names from the file.  
There is a handful of postcodes that even the reference file does not contain; this is because they are special postcodes used for priority distribution to companies (a paid service). We will correct these by way of a specific mapping dictionary once all the other issues are solved.

In [10]:
from difflib import SequenceMatcher
import csv

city_to_pc_map = {
     u'78170': '78170',
     u'Bougival': '78380',
     u'Buc': '78530',
     u'Croissy-sur-Seine': '78290',
     u'Guyancourt': '78280',
     u'La Celle-Saint-Cloud': '78170',
     u'Le Chesnay': '78150',
     u'le Chesnay': '78150',
     u'Le V\xe9sinet': '78110',
     u'Marly-le-Roi': '78160',
     u'Montesson': '78360',
     u'Montigny-le-Bretonneux': '78180',
     u'Noisy-le-Roi': '78590',
     u'Roquencourt': '78150',
     u"Saint-Cyr-l'\xc9cole": '78210',
     u'Saint-Germain-en-Laye': '78100',
     u'Versailles': '78000',
     u'Viroflay': '78220'}

pc_to_city_map = {
    '78103': 'St Germain En Laye',
    '78101': 'St Germain En Laye',
    '78884': 'St Quentin En Yvelines',
    '92852': 'Rueil Malmaison'
}

laposte_file = './Reference/laposte_hexasmal.csv'

def parse_reference_file(filename):
    ''' Take the reference file from La Poste and parse into a dictionaty with 
    postcode as keys. Several towns can have the same postcode, therefore each 
    dict value is a list.
    
    In: 
        filename (string): Path to the reference file to parse
    Out:
        dict of lists: Keys = postcodes, Values = lists of city names associated
            to the key postcode.
    '''    
    data = defaultdict(list)
    with open(filename, 'r') as f:
        reader = csv.reader(f, delimiter = ';') # French CSVs use ; separators
        reader.next()
        for row in reader:
            data[str(row[2])].append(row[1].title())  # Use titlecase everywhere
            # for consistency
    return data
    
def get_pc_from_city(city):
    ''' Returns postcode given city name, using the manually-defined 
    "city_to_pc_map" mapping'''
    return city_to_pc_map[city]

def get_city_from_pc(pc, city, reference):
    ''' Return a correct city name given postcode and a possibly incorrect city 
    name, using the reference file. Because a postcode key can have several city
    values in the reference file, we return the city name with the highest 
    similarity to the city name that we are trying to check/correct.
    
    In:
        pc (string): Postcode for the PC / City combination we are assessing
        city (string): City name for the same.
        reference (dict of lists): Reference data parsed from the reference file
            where keys = postcodes and values = lists of city names associated
            with the key postcode.
    Out:
        string: Corrected city name.
    '''
    if city:
        best_match = [None, 0.] 
        # Initialise: best_match[0] is for the best-matching city name, 
        # best_match[1] is for the match ratio of best_match[0] with the 'city' 
        # string.
        for c in reference[pc]:  # For each city name associated with this 
                                 # particular postcode:
            match_ratio = SequenceMatcher(None, c, city.title()).ratio()  
            # Calculate similarity
            if match_ratio >= best_match[1] or best_match[1] == 0:
                best_match[0] = c  # This is our new best match
                best_match[1] = match_ratio
        #print 'City present:', city, 'Returning best match:', best_match[0]
        return best_match[0]
    else:  # If there is a postcode but no city name in the OSM data, we pick 
           # the first city name associated with this postcode in the reference 
           # dictionary.
        try:          
            reference[pc][0]  
            # print 'No city: Returning first value:', reference[pc][0]
            return reference[pc][0] 
        except IndexError:
            # print 'No city found for postcode', pc
            return
        
def correct_pc_city(pc, city, reference):
    ''' Given a postcode and a city name (both potentially incorrect, and one or the
    other can be missing -- note that if both are missing, this function is not 
    even called):
    - Gets the correct postcode from the city name using get_pc_from_city() + 
    one "manual" adjustment to correct a typo in the data
    - Once all cities have the correct postcode, fix all city names by applying
    get_city_from_pc(). This ensures all city names are correct and follow the
    same spelling and case conventions.
    - Finally, correct four cases where special delivery postcodes are used, 
    which cannot be found in the reference data. This uses the pc_to_city_map
    dictionary.

    In:
        pc (string): Postcode, potentially None or incorrect
        city (string): City name, potentially None or incorrect
        reference (dict of lists): Reference data parsed from the reference file
            where keys = postcodes and values = lists of city names associated
            with the key postcode.
    Out:
        tuple of strings: Corrected postcode, corrected city name
    '''

    '''First step: ensure each city has a valid postcode. '''
    if pc is None: 
        new_pc = get_pc_from_city(city)  # Use the city name to get the postcode
        #print 'No Postcode, returning value from map:', new_pc
    else:
        if pc == '78530' and city == 'Jouy-en-Josas':  # We found one typo where
            # two postcodes got confused for one another
            new_pc = '78350'
        else:
            new_pc = pc  # In the general case keep the same postcode.
        #print 'Postcode present, returning same value (except Jouy en Josas):' \
        #, new_pc
    
    ''' Second step: ensure all postcodes have a correctly formatted city:'''
    new_city = get_city_from_pc(new_pc, city, reference)

    ''' Third step: correct the last four cases where postcode is 
    missing from the reference file, using our second manually-defined 
    mapping.'''
    if new_pc in pc_to_city_map:
        new_city = pc_to_city_map[new_pc]
        
    return new_pc, new_city

            
ref_data = parse_reference_file(laposte_file)

pc_city = set()
for pc, city in pc_cities:  # Apply correct_pc_city() to all postcode/city 
    # combinations found in the OSM XML file:
    new_pc, new_city = correct_pc_city(pc, city, ref_data)
    #print 'OUTCOME: ', new_pc, new_city, '\n'
    pc_city.add((new_pc, new_city))
    
pprint.pprint(pc_city)  # Outcome of our manipulations


set([('78000', 'Versailles'),
     ('78100', 'St Germain En Laye'),
     ('78101', 'St Germain En Laye'),
     ('78103', 'St Germain En Laye'),
     ('78110', 'Le Vesinet'),
     ('78112', 'Fourqueux'),
     ('78120', 'Sonchamp'),
     ('78140', 'Velizy Villacoublay'),
     ('78150', 'Le Chesnay'),
     ('78150', 'Rocquencourt'),
     ('78160', 'Marly Le Roi'),
     ('78170', 'La Celle St Cloud'),
     ('78180', 'Montigny Le Bretonneux'),
     ('78190', 'Trappes'),
     ('78210', 'St Cyr L Ecole'),
     ('78220', 'Viroflay'),
     ('78230', 'Le Pecq'),
     ('78240', 'Chambourcy'),
     ('78280', 'Guyancourt'),
     ('78290', 'Croissy Sur Seine'),
     ('78330', 'Fontenay Le Fleury'),
     ('78350', 'Jouy En Josas'),
     ('78350', 'Les Loges En Josas'),
     ('78360', 'Montesson'),
     ('78380', 'Bougival'),
     ('78390', 'Bois D Arcy'),
     ('78400', 'Chatou'),
     ('78420', 'Carrieres Sur Seine'),
     ('78430', 'Louveciennes'),
     ('78530', 'Buc'),
     ('78560', 'Le Port Mar

Note that this process loses any accented characters as well as apostrophes and hyphens. This is because the La Poste file is meant to be a reference for handscript reading devices and therefore tends to simplify spelling as much as possible by using upper-cases (in French, accents are optional on upper-case characters). We believe it to be an acceptable trade-off for the improved quality of the data.

## B. Investigating the data with the MongoDB API
### B.1. Data load into a MongoDB database

Now that we developped functions to clean up the data, we need to shape it in a way that can be inserted into a MongoDB database. The general schema that we want to use is the same as in the Udacity case study, with one exception: We will hold latitude and longitude separately rather than in the same array, as this will come in handy later on.

Most of the code used here has been written as part of the aforementioned case study, however we need to adapt it in order to include the various checks and corrections developped above. These functions are applied at each iteration just before writing it to the JSON file, as opposed to applying all corrections in one go to the entire data. This approach allows us to use iterative parsing on the OSM XML file. Although the data would easily fit into machine memory, this makes the code re-usable for larger pieces of data.

In [11]:
import codecs
import json


lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')  # Regex for two pieces
# of lowercase strings separated by a colon character
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')  # Regex for any
# potentially problematic character

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]  
# Keys to retain in the 'created' field

def process_tags(tag, tag_id):
    ''' Parses node tags and way tags as per convention. Assumes no problematic 
    characters.
    
    In:
        tag (XML element): A node or way's "tag" sub-element from the OSM XML
            data
        tag_id (string): The node or way's "id" attribute
    Out:
        dict: Contains the following key:value pairs:
            'id': node or way id
            'key': "tag" sub-element's key (anything after the colon in the 
                'k' attribute)
            'value': "tag" sub-element's value ('v' attribute)
            'type': "tag" sub-element's type (anything before the colon in the 
                'k' attribute) if there is a colon, None otherwise
    '''
    k = tag.get('k').strip()
    v = tag.get('v').strip()
    if re.match(lower_colon, k):  # If there is a ':' in the tag key:
        tag_type, tag_key = k.split(':', 1)  # tag_type is anything before ':', 
                                             # tag_key is anything after
        tag_value = v
        if tag_type == "addr":
            tag_type = "address"  # Use correct spelling
    else:   # If there isn't a ':' in the tag key:
        tag_type = None  # Returns None as the tag type
        tag_key = k
        tag_value = v
    return  {'id': tag_id, 'key': tag_key, 'value': tag_value, 'type': tag_type}
    # Return as a dict for convenience
    
def shape_element(element):
    ''' Converts an XML element to the selected JSON schema.
        
    In:
        element (XML element): An element from the OSM XML datafile.
    Out:
        dict: A dictionary with keys and values as per the defined JSON schema.
    '''
    node = {}
    if element.tag == "node" or element.tag == "way" :
        ''' The following values are pulled directly from the XML data:'''
        node['id'] = element.get('id')
        node['type'] = element.tag
        node['visible'] = element.get('visible')
        node['created'] = {n: element.get(n) for n in CREATED} 
        # 'created' is a dict of dicts
        try:
            node['lat'] = float(element.get('lat'))  # Convert coordinates to 
                                                # floats if not already the case
            node['lon'] = float(element.get('lon')) 
        except TypeError:
            node['lat'] = element.get('lat')
            node['lon'] = element.get('lon')
        
        '''"Tag" sub-elements require specific treatment:'''
        for t in element.findall('tag'):  # For each sub-element tagged "tag":
            if not re.match(problemchars, t.get('k')) and \
                len(re.findall(':', t.get('k'))) <= 1: 
            # (i.e. ignore when there are problem chars or more than one ':')
                tag_content = process_tags(t, element.get('id'))  
                # Parse tag and extract id, key, value and type
                if tag_content['type'] is not None:  
                # ie. if the 'k' field contained a colon character
                    if (tag_content['type'] not in node.keys()) or \
                        not (isinstance(node[tag_content['type']], dict)):
                    # Initialise an empty dict if there is no key of this 'type'
                    # or its value is not already a dict
                        node[tag_content['type']] = {}
                    node[tag_content['type']][tag_content['key']] = \
                        tag_content['value']
                    # for instance: node['address']['postcode'] = '78100'
                else: # ie. if the 'k' field didn't contain a colon character
                    node[tag_content['key']] = tag_content['value']
        
        ''' For "nd" tags, we collect all nodes contained in the way:'''
        for t in element.findall('nd'):  
            if 'node_refs' not in node.keys():
                node['node_refs'] = []
            node['node_refs'].append(t.get('ref'))
                    
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    ''' Iterates through the OSM file and saves it into a correctly formatted JSON 
    file, applying cleaning procedures along the way.
    
    In:
        file_in (string): Path to OSM XML file to clean up and convert to JSON
        pretty (bool): If True, add whitespaces as required to the JSON file to
            ensure a pretty formatting. False (default) for large data as 
            prettyfying is expansive.
    Out:
        list of dicts: JSON-formatted data, identical to the data saved to disk.
    '''
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)  # Create a properly shaped JSON-type 
                                         # element as per defined schema.
            if el:           
                ''' Apply cleaning procedures to the address fields: 
                street names, postcodes and cities: '''
                if 'address' in el: 
                    if 'street' in el['address']:  # Clean up street names
                        el['address']['street'] = \
                            update_street_name(
                                el['address']['street'], street_mapping
                                              )
                    if 'postcode' in el['address'] or 'city' in el['address']:  
                    # Clean up postcodes & cities
                    # We ensure either postcode or city (or both) is present
                        try:
                            pc = el['address']['postcode']  # Postcode present
                        except KeyError:
                            pc = None  # Postcode is absent
                        try:
                            city = el['address']['city']  # City present
                        except KeyError:
                            city = None  # City is absent
                        el['address']['postcode'], el['address']['city'] = \
                            correct_pc_city(pc, city, ref_data) 
                        # Apply clean-up function to postcode and city name
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")  # Prefered for large 
                    # datasets as prettyfying the JSON file is expansive
    return data

data = process_map('./Versailles.osm/Versailles.osm', False)

Let us look at a sample from the converted data:

In [12]:
from random import sample, seed

seed(123)
pprint.pprint(sample(data, 5))


[{'created': {'changeset': '5055234',
              'timestamp': '2010-06-22T21:58:33Z',
              'uid': '11699',
              'user': 'wagner51',
              'version': '1'},
  'id': '783430429',
  'lat': 48.7628897,
  'lon': 2.1218889,
  'type': 'node',
  'visible': None},
 {'created': {'changeset': '5375445',
              'timestamp': '2010-08-01T21:38:37Z',
              'uid': '158826',
              'user': 'cquest',
              'version': '1'},
  'id': '842552161',
  'lat': 48.8366723,
  'lon': 2.1085662,
  'type': 'node',
  'visible': None},
 {'created': {'changeset': '8050159',
              'timestamp': '2011-05-04T16:20:34Z',
              'uid': '185687',
              'user': 'apollinaire',
              'version': '1'},
  'id': '1272175173',
  'lat': 48.871806,
  'lon': 2.086614,
  'type': 'node',
  'visible': None},
 {'created': {'changeset': '5478160',
              'timestamp': '2010-08-13T00:55:31Z',
              'uid': '178790',
              'user': 'ahm

The data seems correctly converted to JSON. We now need to load it into a MongoDB database. This is achieved with the `mongoimport` command from the OS command line. The data takes about 30 seconds to load and we get the following prompt:

### B.2. Data investigations with `pymongo`
We will use `pymongo` to query the database from Python. We therefore connect to the database using `MongoClient`.

In [13]:
def get_db():
    ''' Creates connection to local MongoDB database.
    '''
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client.OpenStreetMap
    return db

db = get_db()

def aggregate(query):
    ''' Runs a MongoDB aggregation query on the connected database.

    In:
        query (string): An query using MongoDB's aggregation framework
    Out:
        list: List of documents returned by the query
    '''    
    return [doc for doc in db.Versailles.aggregate(query)]


#### B.2.a. General statistics
* **File size:**  
From the screen capture above, we see that the uncompressed data is 281MB once converted to JSON. Thus, the conversion from OSM XML caused a 25% increase in file size.  


* **Number of documents:**  
Again, we can get this information from the screen capture above, but also programmatically with the following query:

In [14]:
doc_count = [{"$group": {"_id": "Number of documents", "count":{"$sum": 1}}}]
pprint.pprint(aggregate(doc_count))

[{u'_id': u'Number of documents', u'count': 1167432}]


Alternatively, we could have just used a combination of .find() and .count() to get to the same result.

* **Number of nodes:**  
We use the following query:

In [15]:
node_count = [{"$match": {"type": "node"}},
              {"$group": {"_id": "$type", "count": {"$sum": 1}}}]
pprint.pprint(aggregate(node_count))

[{u'_id': u'node', u'count': 1007491}]


* **Number of ways:**  
We use:

In [16]:
way_count = [{"$match": {"type": "way"}},
             {"$group": {"_id": "$type", "count": {"$sum": 1}}}]
pprint.pprint(aggregate(way_count))

[{u'_id': u'way', u'count': 159877}]


#### B.2.b. Specific statistics

* **Number of unique users who edited the data:**  

In [17]:
unique_uids = [{"$group": {"_id": "$created.uid", "count": {"$sum": 1}}},
               {"$group": {"_id": "Unique users", "count": {"$sum": 1}}}]
pprint.pprint(aggregate(unique_uids))

[{u'_id': u'Unique users', u'count': 1014}]


* **Top 10 contributors:**  

In [18]:
top10_users = [{"$group": {"_id": "$created.user", "count": {"$sum": 1}}},
               {"$sort": {"count": -1}},
               {"$limit": 10}]
pprint.pprint(aggregate(top10_users))

[{u'_id': u'wagner51', u'count': 324461},
 {u'_id': u'Marcussacapuces91', u'count': 168536},
 {u'_id': u'osmmaker', u'count': 136707},
 {u'_id': u'didier2020', u'count': 101247},
 {u'_id': u'Dave92', u'count': 44346},
 {u'_id': u'cquest', u'count': 36809},
 {u'_id': u'PierenBot', u'count': 27341},
 {u'_id': u'apollinaire', u'count': 21435},
 {u'_id': u'uploaddidier2020', u'count': 19151},
 {u'_id': u'alexis78', u'count': 18936}]


* **Number of nodes per city (top 10):**  
We only retain "node" elements that contain a city name:

In [19]:
top10_cities = [{"$match": {"address.city": {"$exists": 1}}},
                {"$group": {"_id": "$address.city", "count": {"$sum": 1}}},
                {"$sort": {"count": -1}},
                {"$limit": 10}]
pprint.pprint(aggregate(top10_cities))

[{u'_id': u'St Germain En Laye', u'count': 824},
 {u'_id': u'Versailles', u'count': 754},
 {u'_id': u'Buc', u'count': 487},
 {u'_id': u'Le Vesinet', u'count': 293},
 {u'_id': u'St Cyr L Ecole', u'count': 272},
 {u'_id': u'Marly Le Roi', u'count': 270},
 {u'_id': u'Viroflay', u'count': 135},
 {u'_id': u'Chatou', u'count': 123},
 {u'_id': u'Noisy Le Roi', u'count': 121},
 {u'_id': u'Le Pecq', u'count': 83}]


This ranking is surprising because we would instinctively expect the number of node to be somewhat correlated to the population of each town. Here is the population for these towns, in the same order ([source](http://www.toutes-les-villes.com/villes-departements/78-yvelines/1.html)):

|City / Town:         |Population|
|---------------------|----------|
|St Germain en Laye   |    38,124|
|Versailles           |    85,761|
|Buc                  |     5,743|
|Le Vésinet           |    15,928|
|St Cyr L'Ecole       |    14,585|
|Marly Le Roi         |    16,787|
|Viroflay             |    15,205|
|Chatou               |    28,582|
|Noisy Le Roi         |     7,711|
|Le Pecq              |    16,342|

Of course, the boundaries of the map extract are rectangular and therefore some of the cities will be incomplete. However it is still strange to see that Versailles, the largest city by far (and the most touristic), is only second in the ranking while Buc is number 3 while being the smallest of these 10.
One possible explanation is that some of the most active contributors have focused on a few areas only (typically, where they live).  

Let us test this theory:

In [20]:
'''Cities edited by top 10 contributors:'''

# First we need to retrieve the top 10 users, but only for elements that have a city name:
match = [{"$match": {"address.city": {"$exists": 1}}},
         {"$group": {"_id": "$created.user", "count": {"$sum": 1}}},
               {"$sort": {"count": -1}},
               {"$limit": 10},
         {"$project": {"_id": 1}}]

# Turn the result into a list of users:
user_list = [u[u'_id'] for u in aggregate(match)]
print "Top 10 contributors for elements that contain a city name:"
print user_list

# Use this list to filter down users:
cities_by_user = [{"$match": {"created.user": {"$in": user_list},
                              "address.city": {"$ne": None}}},
                  {"$group": 
                       {"_id": {"user": "$created.user", "city": "$address.city"},
                        "count": {"$sum": 1}}},
                  {"$sort": {"_id.user": 1}}]
print "\nNumber of contribution by these 10 users, by city:"
pprint.pprint(aggregate(cities_by_user))

Top 10 contributors for elements that contain a city name:
[u'feuerbach78', u'osmmaker', u'wagner51', u'Guillaume Castagnino', u'didier2020', u'GeorgeKaplan', u'XavierT', u'Marcussacapuces91', u'Carte-on', u'denis1378']

Number of contribution by these 10 users, by city:
[{u'_id': {u'city': u'Buc', u'user': u'Carte-on'}, u'count': 61},
 {u'_id': {u'city': u'Versailles', u'user': u'Carte-on'}, u'count': 1},
 {u'_id': {u'city': u'Montigny Le Bretonneux', u'user': u'Carte-on'},
  u'count': 1},
 {u'_id': {u'city': u'Guyancourt', u'user': u'GeorgeKaplan'}, u'count': 1},
 {u'_id': {u'city': u'St Cyr L Ecole', u'user': u'GeorgeKaplan'}, u'count': 1},
 {u'_id': {u'city': u'Le Chesnay', u'user': u'GeorgeKaplan'}, u'count': 1},
 {u'_id': {u'city': u'Le Vesinet', u'user': u'GeorgeKaplan'}, u'count': 1},
 {u'_id': {u'city': u'Croissy Sur Seine', u'user': u'GeorgeKaplan'},
  u'count': 1},
 {u'_id': {u'city': u'St Germain En Laye', u'user': u'GeorgeKaplan'},
  u'count': 27},
 {u'_id': {u'city': u'Ch

This list is the number of elements by city, from the top 10 contributors on elements containing city names. We notice several features:
- User "wagner51" contributed in only 3 cities, with 389 edits to Buc (out of a total of 487 for this small town)
- User "feuerbach78" contributed in 7 cities, including St-Germain-en-Laye for 625 edits (out of a total of 824 for this city) and Marly-Le-Roi for 241 edits (out of 270)
- User "didier2020" contributed in 6 cities, most of their edits were against Noisy-Le-Roi where he accounts for 106 out of 121 edits.

It appears that OpenStreetMap being a relatively unknown application (at least in France), there is a very unbalanced distribution in the number of edits per user, with a few users generating the vast majority of the elements. This makes statistical comparisons with the number of inhabitants of a particular city are largely irrelevant.

#### B.2.c. Visiting the Château de Versailles

There is a lot of information that we may want to get from the database about the Versailles palace. We will give a few examples below.  
The first thing to define is what is considered as the palace and its grounds. Looking at the map, we can roughly approximate it with a horizontal, rectangular box between latitudes [48.801217, 48.820209] and longitudes [2.079505, 2.123966].  

So as a tourist, how many points of interest can you expect to see?

In [21]:
tourism_spots = [{"$match": 
                      {"lat": {"$gte": 48.801217},
                       "lat": {"$lte": 48.828209},
                       "lon": {"$gte": 2.079505},
                       "lon": {"$lte": 2.123966},
                       "tourism": {"$exists": 1}
                      }
                 },
                 {"$group": {"_id": "$tourism", "count": {"$sum": 1}}}
                ]

pprint.pprint(aggregate(tourism_spots))

[{u'_id': u'caravan_site', u'count': 2},
 {u'_id': u'picnic_site', u'count': 2},
 {u'_id': u'viewpoint', u'count': 2},
 {u'_id': u'yes', u'count': 1},
 {u'_id': u'information', u'count': 16},
 {u'_id': u'museum', u'count': 4},
 {u'_id': u'hotel', u'count': 7},
 {u'_id': u'attraction', u'count': 14},
 {u'_id': u'artwork', u'count': 204}]


Some these categories are somewhat unspecific. Let us look into "attractions" in more detail:

In [22]:
attractions = [{"$match": 
                      {"lat": {"$gte": 48.801217},
                       "lat": {"$lte": 48.828209},
                       "lon": {"$gte": 2.079505},
                       "lon": {"$lte": 2.123966},
                       "tourism": "attraction"
                      }
                 },
                 {"$project": {"_id": 0,
                               "attraction_name": "$name"}}
                ]

pprint.pprint(aggregate(attractions))

[{u'attraction_name': u'Le hameau de la reine'},
 {u'attraction_name': u'Ferme'},
 {u'attraction_name': u'P\xe9ristyle'},
 {u'attraction_name': u'Trianon-sous-Bois'},
 {u'attraction_name': u'Orangerie'},
 {u'attraction_name': u'Aile Nord'},
 {u'attraction_name': u'Aile Sud'},
 {u'attraction_name': u'Galerie des Cotelle'},
 {u'attraction_name': u'Grand Appartement de la Reine'},
 {u'attraction_name': u'Grand Appartement du Roi'},
 {u'attraction_name': u'Grotte'},
 {u'attraction_name': u'Salles des Croisades'},
 {u'attraction_name': u'solitude'},
 {u'attraction_name': u'Le ch\xeane de Louis XIV'}]


I can confirm that these are all well worth your time -- Some of them are part of the main building, while others are separate buidings or features. The "Hameau de la Reine" for instance is a charming little hamlet that was built to satisfy Queen Marie-Antoinette's fantasies of living a rural life.

![hameau_de_la_reine](img/PA0111-hr.jpg)

The Orangerie is also very nice. It is still used to grow orange trees and other species. They can be carried inside the building in the winter or taken outside in the spring and summer:

![orangerie](img/Garden-orangerie_Exterior_of_the_Palace_of_Versailles.JPG)

Now what about these 204 artworks reported in the first query about the Versailles palace?

In [23]:
artworks = [{"$match": 
                      {"lat": {"$gte": 48.801217},
                       "lat": {"$lte": 48.828209},
                       "lon": {"$gte": 2.079505},
                       "lon": {"$lte": 2.123966},
                       "tourism": "artwork"
                      }
             },
             {"$group": {"_id": "$artwork_type",
                         "count": {"$sum": 1}}}
           ]

pprint.pprint(aggregate(artworks))

[{u'_id': u'statue', u'count': 1},
 {u'_id': u'sculpture', u'count': 183},
 {u'_id': None, u'count': 20}]


184 sculptures and statues! This is actually far from the full collection, as there are over 300 statues in the castle grounds alone. The data is therefore about 60% complete as far as sculptures go.

How about fountains?

In [24]:
fountains = [{"$match": 
                      {"lat": {"$gte": 48.801217},
                       "lat": {"$lte": 48.828209},
                       "lon": {"$gte": 2.079505},
                       "lon": {"$lte": 2.123966},
                       "amenity": "fountain"
                      }
              },
              {"$group": {"_id": "$amenity", "count": {"$sum": 1}}}
            ]

pprint.pprint(aggregate(fountains))

[{u'_id': u'fountain', u'count': 35}]


From [Wikipedia](https://fr.wikipedia.org/wiki/Jardin_de_Versailles) we learn that there are actually 55 fountains on the Versailles castle grounds. So again, the data is about 64% complete.

## C. Ideas for further improvement / exploration

### C.1. Use reference data for street names
To further clean up the data, one could compare street names with a record of all existing street names in the area. Such databases are available but quite expensive (see [here](https://gumroad.com/l/VzKNO) for instance). The cleaning up process would then need to compare each street in the OSM file with this reference, city by city, and compute similarities to pick the most likely street name (like we did for city names earlier in this project).

* **Benefits:**
    - Data is probably exhaustive and accurate
    - Data already in an easily usable format

* **Drawbacks / potential problems:**
    - Cost
    - Data would need to be purchased and processed on a regular basis to remain up-to-date

### C.2. Use data from the Google Maps API
Google Map has very comprehensive data and it is made available through an API. We could use this to enrich our dataset, especially with regards to street names again.

If we wanted to stay consistent our "tourist" approach, we could also pull out a list of all hotels, restaurants and public transportation in the area by using search functions, then include them into our dataset.

* **Benefits:**
    - Reference data is generally high quality and updated regularly

* **Drawbacks / potential problems:**
    - Usage rights?
    - API learning curve?

### C.3. Organise "field days"
We saw that the vast majority of the data is supplied by only a few contributors who tend to focus on small geographical areas. The two obvious ways to increase the number and span of contributions are:

- Get more people to contribute
- Get the heavy contributors to submit data outside of their usual area

To do this, one idea could be to organise field days where OpenStreetMap users could go out in a given area in competing teams. Using a dedicated smartphone app, they would take pictures of street plaques and buildings. These pictures would be then processed by a machine learning system and/or manually commented and, using geo-tagging, would help complete the OSM data.

* **Benefits:**
    - Increase involvement of users and create a real community
    - Allow targetting of the areas to improve
    
* **Drawbacks / potential problems:**
    - Requires a coordination team to organise and manage
    - Investment in smartphone app and rewards

I hope you enjoyed this exploration of the Versailles area and its palace and that you learned some interesting facts beside the technical aspects of the project. 